# TensorFlow + RedisAI

# Redis AI Python Client

https://github.com/RedisAI/redisai-py

https://github.com/RedisAI/redisai-py/blob/master/redisai/client.py

In [ ]:
!pip install redis==3.2.1 redisai==0.2.0 scikit-image==0.14.2

In [ ]:
import six
import json
import time
from redisai.client import Client, Type, DType, Tensor, BlobTensor
from skimage import io
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Setup RedisAI Client
`redis-master:6379` => Internal Kubernetes DNS host:port

In [ ]:
rai = Client(host="redis-master", port=6379)

In [ ]:
inputs_np = np.array([1.0, 2.0, 3.0])

vector_np_tensor = Tensor(DType.double, 
                          inputs_np.shape, 
                          inputs_np.tolist())

rai.tensorset("vector_np", vector_np_tensor)

In [ ]:
rai.tensorget("vector_np")

In [ ]:
rai.tensorget("vector_np").value

In [ ]:
inputs_np = np.array([1.0, 2.0, 3.0])

np_blob_tensor = BlobTensor.from_numpy(inputs_np)

rai.tensorset("np_blob", np_blob_tensor)

In [ ]:
rai.tensorget("np_blob")

In [ ]:
rai.tensorget("np_blob").value

# Setup ImageNet Dataset (Predictions)
[ImageNet Dataset](http://www.image-net.org/)

`imagenet_classes.json`

In [ ]:
prediction_classes = json.load(open("imagenet_classes.json"))

# Load Redis with Trained ResNet50 Model (TensorFlow)
![](https://cdn-images-1.medium.com/max/1600/1*aq0q7gCvuNUqnMHh4cpnIw.png)

In [ ]:
!wget -O resnet50.pb https://github.com/RedisAI/redisai-examples/raw/master/models/imagenet/tensorflow/resnet50.pb

In [ ]:
tf_model_path = 'resnet50.pb'
with open(tf_model_path, 'rb') as f:
    tf_model = f.read()

rai.execute_command('AI.MODELSET', 'imagenet_model', 'TF', 'CPU', 'INPUTS', 'images', 'OUTPUTS', 'output', tf_model)

# Load Redis with Image Processing Utils
`data_processing_script_tensorflow.py`

In [ ]:
script_path = 'data_processing_script_tensorflow.py'
with open(script_path, 'rb') as f:
    script = f.read()
    
rai.execute_command('AI.SCRIPTSET', 'imagenet_script', 'CPU', script)

# Make Predictions Directly in Redis!

In [ ]:
cat_img_path = 'cat.jpg'

cat_image = io.imread(cat_img_path)

cat_image_blob_tensor = BlobTensor(DType.uint8,
                                   cat_image.shape,
                                   cat_image.tobytes())

rai.tensorset("cat_image_blob", cat_image_blob_tensor)
plt.axis('off')
plt.imshow(cat_image)

In [ ]:
rai.tensorget("cat_image_blob")

In [ ]:
rai.tensorget("cat_image_blob").value[100:110]

In [ ]:
start_time = time.time()

rai.execute_command('AI.TENSORSET', 'image', 'UINT8', *cat_image.shape, 'BLOB', cat_image.tobytes())
rai.execute_command('AI.SCRIPTRUN', 'imagenet_script', 'pre_process_3ch', 'INPUTS', 'image', 'OUTPUTS', 'temp1')
rai.execute_command('AI.MODELRUN', 'imagenet_model', 'INPUTS', 'temp1', 'OUTPUTS', 'temp2')
rai.execute_command('AI.SCRIPTRUN', 'imagenet_script', 'post_process', 'INPUTS', 'temp2', 'OUTPUTS', 'out')

end_time = time.time()

prediction = rai.execute_command('AI.TENSORGET', 'out', 'VALUES')

print("Predicted index", prediction[2][0], "in", (end_time - start_time), "seconds")

In [ ]:
idx = prediction[2][0]

print(idx, "=>", prediction_classes[str(idx)])

In [ ]:
guitar_img_path = 'guitar.jpg'

guitar_image = io.imread(guitar_img_path)

guitar_image_blob_tensor = BlobTensor(DType.uint8,
                                      guitar_image.shape,
                                      guitar_image.tobytes())

rai.tensorset("guitar_image_blob", guitar_image_blob_tensor)
plt.axis('off')
plt.imshow(guitar_image)

In [ ]:
rai.tensorget("guitar_image_blob")

In [ ]:
rai.tensorget("guitar_image_blob").value[400:410]

In [ ]:
start_time = time.time()

rai.execute_command('AI.TENSORSET', 'image', 'UINT8', *guitar_image.shape, 'BLOB', guitar_image.tobytes())
rai.execute_command('AI.SCRIPTRUN', 'imagenet_script', 'pre_process_3ch', 'INPUTS', 'image', 'OUTPUTS', 'temp1')
rai.execute_command('AI.MODELRUN', 'imagenet_model', 'INPUTS', 'temp1', 'OUTPUTS', 'temp2')
rai.execute_command('AI.SCRIPTRUN', 'imagenet_script', 'post_process', 'INPUTS', 'temp2', 'OUTPUTS', 'out')

end_time = time.time()

prediction = rai.execute_command('AI.TENSORGET', 'out', 'VALUES')

print("Predicted index", prediction[2][0], "in", (end_time - start_time), "seconds")

In [ ]:
idx = prediction[2][0]

print(idx, "=>", prediction_classes[str(idx)])

In [ ]:
salvatore_img_path = 'salvatore.jpg'

salvatore_image = io.imread(salvatore_img_path)

salvatore_image_blob_tensor = BlobTensor(DType.uint8,
                                         salvatore_image.shape,
                                         salvatore_image.tobytes())

rai.tensorset("salvatore_image_blob", salvatore_image_blob_tensor)

plt.axis('off')
plt.imshow(salvatore_image)

In [ ]:
rai.tensorget("salvatore_image_blob")

In [ ]:
rai.tensorget("salvatore_image_blob").value[400:410]

In [ ]:
start_time = time.time()

rai.execute_command('AI.TENSORSET', 'image', 'UINT8', *salvatore_image.shape, 'BLOB', salvatore_image.tobytes())
rai.execute_command('AI.SCRIPTRUN', 'imagenet_script', 'pre_process_3ch', 'INPUTS', 'image', 'OUTPUTS', 'temp1')
rai.execute_command('AI.MODELRUN', 'imagenet_model', 'INPUTS', 'temp1', 'OUTPUTS', 'temp2')
rai.execute_command('AI.SCRIPTRUN', 'imagenet_script', 'post_process', 'INPUTS', 'temp2', 'OUTPUTS', 'out')

end_time = time.time()

prediction = rai.execute_command('AI.TENSORGET', 'out', 'VALUES')

print("Predicted index", prediction[2][0], "in", (end_time - start_time), "seconds")

In [ ]:
idx = prediction[2][0]

print(idx, "=>", prediction_classes[str(idx)])